In [1]:
import pandas as pd

In [2]:
y_train = pd.read_csv('/workspaces/EDA_4/data/processed/heart_prevalence_y_train.csv')
y_test = pd.read_csv('/workspaces/EDA_4/data/processed/heart_prevalence_y_test.csv')
X_train_norm = pd.read_csv('/workspaces/EDA_4/data/interim/heart_prevalence_X_train_std.csv')
X_test_norm = pd.read_csv('/workspaces/EDA_4/data/interim/heart_prevalence_X_test_std.csv')

In [3]:
from sklearn.feature_selection import f_regression, SelectKBest

In [4]:
selection_model = SelectKBest(f_regression, k = 12)
selection_model.fit(X_train_norm, y_train)
ix = selection_model.get_support()
X_train_sel = pd.DataFrame(selection_model.transform(X_train_norm), columns = X_train_norm.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test_norm), columns = X_test_norm.columns.values[ix])

/home/vscode/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
X_train_sel["Heart disease_prevalence"] = y_train
X_test_sel["Heart disease_prevalence"] = y_test 

In [6]:
X_train_sel.to_csv("/workspaces/EDA_4/data/interim/heart_prevalence_X_train_sel.csv", index=False)
X_test_sel.to_csv("/workspaces/EDA_4/data/interim/heart_prevalence_X_test_sel.csv", index=False)

In [7]:
train_data = X_train_sel
test_data = X_test_sel
train_data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,Heart disease_prevalence
0,-1.142601,-0.758344,-0.524870,1.724298,2.504393,0.820420,-0.393446,-1.254632,-1.034310,-0.690028,-1.329247,-1.063396,7.321739
1,1.195765,0.657546,0.402394,0.477422,0.081398,0.113865,0.558434,-0.126974,-0.141938,-0.158183,-0.433800,-0.535918,8.031343
2,-0.733491,-0.205473,-0.192600,-0.769453,-0.600070,-0.311410,0.579479,0.363973,0.936350,-0.307709,0.011809,-0.196489,8.315238
3,1.083623,1.001405,1.283295,1.133672,1.141458,1.409531,0.635620,-1.011609,-1.099331,-0.350395,-1.043889,-1.534241,7.230000
4,-0.376298,0.003540,-0.068965,-0.178828,0.157116,-1.292706,0.882003,0.003443,-0.337835,-0.487564,0.924241,1.998352,8.663636


In [8]:
X_train = train_data.drop(["Heart disease_prevalence"], axis = 1)
y_train = train_data["Heart disease_prevalence"]
X_train.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,-1.142601,-0.758344,-0.524870,1.724298,2.504393,0.820420,-0.393446,-1.254632,-1.034310,-0.690028,-1.329247,-1.063396
1,1.195765,0.657546,0.402394,0.477422,0.081398,0.113865,0.558434,-0.126974,-0.141938,-0.158183,-0.433800,-0.535918
2,-0.733491,-0.205473,-0.192600,-0.769453,-0.600070,-0.311410,0.579479,0.363973,0.936350,-0.307709,0.011809,-0.196489
3,1.083623,1.001405,1.283295,1.133672,1.141458,1.409531,0.635620,-1.011609,-1.099331,-0.350395,-1.043889,-1.534241
4,-0.376298,0.003540,-0.068965,-0.178828,0.157116,-1.292706,0.882003,0.003443,-0.337835,-0.487564,0.924241,1.998352


In [9]:
X_test = test_data.drop(["Heart disease_prevalence"], axis = 1)
y_test = test_data["Heart disease_prevalence"]
X_test.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,2.556004,1.695866,1.128751,1.068047,0.308553,2.333274,-0.914804,-0.312357,-0.103458,-0.745705,-0.220811,-1.029929
1,12.129598,10.022651,7.449600,11.371176,6.138886,3.201860,-3.699774,-2.592387,-2.434579,-2.193322,-2.465568,-2.489778
2,0.840649,1.237387,1.391476,0.805547,0.914302,-0.023037,1.816672,-0.233735,-0.412420,0.063166,-0.085710,0.403377
3,0.473073,0.293460,0.456484,-0.113203,0.005679,0.546627,0.280258,-1.980160,-2.283529,-1.204175,-1.573678,-0.723029
4,1.397205,2.552143,2.836462,2.708673,3.488735,-0.386932,1.693048,-0.434702,-0.682066,0.068579,-0.628342,-0.081028


In [10]:
from sklearn.linear_model import Ridge

# Load of train and test data
# These data must have been standardized and correctly processed in a complete EDA

ridge_model = Ridge(alpha = 0.1, max_iter = 300)

ridge_model.fit(X_train, y_train)

y_pred = ridge_model.predict(X_test)



In [11]:
from sklearn.metrics import mean_squared_error, r2_score
import math

print(f"Error cuadrático medio: {math.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred)}")

Error cuadrático medio: 0.2652861691878653
Coeficiente de determinación: 0.9665416162946575
